# Setup Mimic
- Download Mimic
- unzip Mimic
- load into postgresql database

In [8]:
import re

import numpy as np
import pandas as pd
from IPython.display import display
from sql_alchemy_utility.sql_alchemy_utility import SqlOperations as Sql

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2023-07-01 18:45:39 -04:00)


In [6]:
# test the sql query
test_query: str = "SELECT * FROM noteevents LIMIT 10"
test_df = Sql.load_data_from_db(test_query)
test_df

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,174,22532,167853,2151-08-04,None,None,Discharge summary,Report,None,None,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,None,None,Discharge summary,Report,None,None,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,None,None,Discharge summary,Report,None,None,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,None,None,Discharge summary,Report,None,None,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,None,None,Discharge summary,Report,None,None,Admission Date: [**2162-3-3**] D...
5,179,53181,170490,2172-03-08,None,None,Discharge summary,Report,None,None,Admission Date: [**2172-3-5**] D...
6,180,20646,134727,2112-12-10,None,None,Discharge summary,Report,None,None,Admission Date: [**2112-12-8**] ...
7,181,42130,114236,2150-03-01,None,None,Discharge summary,Report,None,None,Admission Date: [**2150-2-25**] ...
8,182,56174,163469,2118-08-12,None,None,Discharge summary,Report,None,None,Admission Date: [**2118-8-10**] ...
9,183,56174,189681,2118-12-09,None,None,Discharge summary,Report,None,None,Admission Date: [**2118-12-7**] ...


In [4]:
## dictionaries to contain regular expression hints to find keywords for VTE interventions and negations

# common regex patterns
common_bt_reg = 'anticoagula(?:nt|tion)|bood thinner|antiplatelets'
anticoagulant_reg = 'anticoagul|aspirin|lovenox|enoxaparin|heparin'
blood_thinner_reg = 'argatroban|dabigatran|warfarin|jantoven|coumadin'
dti_reg = 'direct thrombin inhibitor|refludan|lepirudin|bivalirudin'
gp_inhibitor_reg = 'iib/iiia inhibitor|abciximab|integrilin,eptifibatide|tirofiban,plavix,clopidogrel'
xa_inhibitor_reg = 'xa inhibitor|xarelto|rivaroxaban|eliquis|apixaban|arixtra|fondaparinux|savaysa|edoxaban|bevyxxa|betrixaban'
phys_reg = 'foot pumps|pneumatic compression|compression stockings|sequential compression|ambulation|leg exercise|early mobilization'

refuse_reg = '(?:avoid|declin|refus)(?:e[sd]?|ing)?'
allergy_reg = 'allerg(?:i(?:es|c)|y)'
neg_reaction_reg = f'(?:(?<!positive)) (?:serious|negative|{allergy_reg})? ?(?:interaction|reaction|{allergy_reg}) (?:with|to)?'

# hints for VTE interventions
# portal vein thrombosis
# thrombin inhibitor therapy
# thrombi
# proximal thrombus
# DVT prophylaxis: mechanical
# pneumoboots
regEx_inter = {
    '(\Wvte\W|venous thromboembolism)' : 'venous thromboembolism',
    '(\Wdvt\W|deep vein thrombosis|deep venous thrombosis)' : 'deep vein thrombosis',
    '(pulmonary embolism|pulmonary arterial embolism)' : 'pulmonary embolism',
    '(\Wcvt\W|cerebral venous thrombosis)' : 'cerebral venous thrombosis',
    '(arterial (?:thrombo)?embolism)' : 'arterial thromboembolism',
    '(\Wdic\W|disseminated intravascular coagulation )' : 'disseminated intravascular coagulation ',
    '(thromboembolic|thrombotic)' : 'thromboembolic',
    '(thromb(?:oembolus|us|os)|blood clot)' : 'thrombus|embolus|blood clot',
    '(superior vena cava obstruction)' : 'superior vena cava obstruction',
    '(atrial fibrillation)' : 'atrial fibrillation',
    '(d-dimer)' : 'd-dimer',
    '(duplex ultrasound)' : 'duplex ultrasound',
    '(ct|computed tomography|pulmonary) angiography' : 'angiography',
    '(troponin)' : 'troponin',
    '(thrombectomy)' : 'thrombectomy',
    '(prothrombin time)' : 'prothrombin time',
    '(\Winr\W|international normalized ratio)' : 'international normalized ratio',
    '(\Wppx\W|prophylaxis|prophylactic)' : 'prophylaxis',
    '(dvt prophylaxis)' : 'dvt prophylaxis',
    '(\Wcabg\W|\Wpci\W|coronary artery bypass grafting|percutaneous coronary intervention)' : 'CABG|PCI',
    '(\Wivc\W|inferior vena cava|greenfield filter)' : 'inferior vena cava',
    '(thrombolytic| tpa |tissue plasminogen activator)' : 'thrombolytic therapy|tissue plasminogen activator',
    f'({blood_thinner_reg})' : f'{blood_thinner_reg}',
    f'({anticoagulant_reg})' : f'{anticoagulant_reg}',
    f'({dti_reg})' : f'{dti_reg}',
    f'({gp_inhibitor_reg})' : f'{gp_inhibitor_reg}',
    f'({xa_inhibitor_reg})' : f'{xa_inhibitor_reg}',
    f'({phys_reg})' : 'prophylaxis device',
    '((?:hip|knee) replacement)' : 'hip|knee replacement',
    '(surgery|procedure)' : 'surgery|procedure',
    '(\Wicu\W)' : 'denEx:ICU',
    '(\Wscip\W|surgical care improvement project)' : 'denEx:surgical care improvement project',
    '(mental disorder)' : 'denEx:mental disorder',
    '(stroke)' : 'denEx:stroke',
    '(intervention comfort measures)' : 'denEx:intervention comfort measures'
}

# hints for VTE negations
# heparin dependent antibodies
# no heparin to be administered
# heparin induced antibodies
# Heparin induced thrombocytopenia antibody was checked and this was negative
# Heparin and other anticoagulants were held
# refuses sc heparin
# refused IV heparin
# The patient refused heparin products and anticoagulation for this aneurysm due to prior GI bleed while anticoagulated
# has a warfarin allergy
# the dropping platelet count, we had to stop the anticoagulation. she currently cannot be anticoagulated.
# IVC filter placed instead of anticoagulation due to her thrombocytopenia and risk of bleeding. Pt declined
# not receiving any anticoagulation given low platelets
# recommended that she not be on anticoagulation
regEx_neg = {
    '((?:(?<!no ))contraindication|contraindicate|negation)' : 'contraindication|negation',
    '((?:thrombolytic|anticoagulant) (?:was|is) (?:contraindicated|negated))' : 'thrombolytic contraindication|negation',
    f'((?:patient|pt|he|she) (?:ha[s|d])? ?{refuse_reg})' : 'refuse|decline',
    f'({refuse_reg} (?:dvt|vte)? ?(?:ppx|prophylaxis))' : 'refuse prophylaxis',
    f'({refuse_reg} thrombocytopenia)' : 'refuse thrombocytopenia',
    f'({refuse_reg} (?:{common_bt_reg}|{anticoagulant_reg}))' : f'refuse {common_bt_reg}|{anticoagulant_reg}',
    f'({refuse_reg} (?:{blood_thinner_reg}))' : f'refuse {blood_thinner_reg}',
    f'({refuse_reg} (?:{dti_reg}))' : f'refuse {dti_reg}',
    f'({refuse_reg} (?:{gp_inhibitor_reg}))' : f'refuse {gp_inhibitor_reg}',
    f'({refuse_reg} (?:{xa_inhibitor_reg}))' : f'refuse {xa_inhibitor_reg}',
    f'({refuse_reg} (?:{phys_reg}))' : 'refuse physical prophylaxis',
    '((?:did|could)(?: not|n\Wt|nt) tolerate)' : 'did not tolerate',
    f'({allergy_reg})' : 'allergies',
    f'({neg_reaction_reg})' : 'negative reaction',
    f'({neg_reaction_reg} (?:{common_bt_reg}|{anticoagulant_reg}))' : f'negative reaction to {anticoagulant_reg}',
    f'({neg_reaction_reg} (?:{blood_thinner_reg}))' : f'negative reaction to {blood_thinner_reg}',
    f'({neg_reaction_reg} (?:{dti_reg}))' : f'negative reaction to {dti_reg}',
    f'({neg_reaction_reg} (?:{gp_inhibitor_reg}))' : f'negative reaction to {gp_inhibitor_reg}',
    f'({neg_reaction_reg} (?:{xa_inhibitor_reg}))' : f'negative reaction to {xa_inhibitor_reg}',
    f'({neg_reaction_reg} (?:{phys_reg}))' : 'negative reaction to physical prophylaxis',
    '(induced thrombocytopenia)' : 'induced thrombocytopenia',
    '(not a candidate for (?:a|the)? ?(?:thrombolytic|anticoagulation))' : 'non-candidate thrombolytic'

}

def applyRegEx(_df: pd.DataFrame, textCol: str) -> pd.DataFrame:
    """
    return a dataframe with the applied regex to the text column
    """
    # for each regex pattern, extract all matching capture groups as one string
    _df["regEx_interv"] = ''
    for key, value in regEx_inter.items():
        _df['intr:' + value] = _df[textCol].str.extractall(key, flags=re.IGNORECASE).groupby(level=0).agg(','.join)
        _df["regEx_interv"] = np.where(_df['intr:' + value].notnull(), _df["regEx_interv"] + ',' + value, _df["regEx_interv"])

    _df["regEx_neg"] = ''
    for key, value in regEx_neg.items():
        _df['neg:' + value] = _df['text'].str.extractall(key, flags=re.IGNORECASE).groupby(level=0).agg(','.join)
        _df["regEx_neg"] = np.where(_df['neg:' + value].notnull(), _df["regEx_neg"]+',' + value, _df["regEx_neg"])
 
    # clean up of new columns by removing initial ','
    _df["regEx_interv"] = _df["regEx_interv"].map(lambda x: x[1:])
    _df["regEx_neg"] = _df["regEx_neg"].map(lambda x: x[1:])

    # clean up of new columns by placeing '' with nan
    _df.loc[_df["regEx_interv"] == '', 'regEx_interv'] = np.nan
    _df.loc[_df["regEx_neg"] == '', 'regEx_neg'] = np.nan

    return _df

#test the regex
values = [
     (' thrombolytic is contraindicated',)
    ,(' contraindication to medicine ',)
    ,(' no contraindication to medicine ',)
    ,(' positive reaction to warfarin ',)
    ,(' negative reaction to warfarin ',)
    ,(' Allergies Allergen Reactions Warfarin Other. allergy to coumadin ',)
    ,(' Allergic to Warfarin ',)
    ,(' enoxaparin ',)
    ,(" he couldn't tolerate ",)
    ,(' heparin ',)
    ,(' patient declined enoxaparin ',)
    ,(' patient declined treatment ',)
    ,(' patient did refuse to enoxaparin ',)
    ,(' patient refuses lovenox ',)
    ,(' pt did not tolerate. drug is contraindicated ',)
    ,(' refusal to ppx ',)
    ,(' serious reaction to venous foot pumps ',)
    ,(' She declined routine tele and dvt ppx ',)
    ,(' She did decline pe ppx ',)
    ,(' right patella tendon. The patient tolerated the procedure well. ',)
    ,(' She refuses vte /vte ppx ',)
    ,(' He had thromboembolus thrombus',)
]

# create pandas dataframe from the test data
df_ex = pd.DataFrame(values, columns=['text'])
display(applyRegEx(df_ex, 'text'))

,text,regEx_interv,intr:venous thromboembolism,intr:deep vein thrombosis,intr:pulmonary embolism,intr:cerebral venous thrombosis,intr:arterial thromboembolism,intr:disseminated intravascular coagulation,intr:thromboembolic,intr:thrombus|embolus|blood clot,...,neg:allergies,neg:negative reaction,neg:negative reaction to anticoagul|aspirin|lovenox|enoxaparin|heparin,neg:negative reaction to argatroban|dabigatran|warfarin|jantoven|coumadin,neg:negative reaction to direct thrombin inhibitor|refludan|lepirudin|bivalirudin,"neg:negative reaction to iib/iiia inhibitor|abciximab|integrilin,eptifibatide|tirofiban,plavix,clopidogrel",neg:negative reaction to xa inhibitor|xarelto|rivaroxaban|eliquis|apixaban|arixtra|fondaparinux|savaysa|edoxaban|bevyxxa|betrixaban,neg:negative reaction to physical prophylaxis,neg:induced thrombocytopenia,neg:non-candidate thrombolytic
0,thrombolytic is contraindicated,thrombolytic therapy|tissue plasminogen activator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,contraindication to medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,no contraindication to medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,positive reaction to warfarin,argatroban|dabigatran|warfarin|jantoven|coumadin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,negative reaction to warfarin,argatroban|dabigatran|warfarin|jantoven|coumadin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,negative reaction to,NaN,negative reaction to warfarin,NaN,NaN,NaN,NaN,NaN,NaN
5,Allergies Allergen Reactions Warfarin Other. ...,argatroban|dabigatran|warfarin|jantoven|coumadin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Allergies,allergy","Allergies , allergy to",NaN,allergy to coumadin,NaN,NaN,NaN,NaN,NaN,NaN
6,Allergic to Warfarin,argatroban|dabigatran|warfarin|jantoven|coumadin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Allergic,Allergic to,NaN,Allergic to Warfarin,NaN,NaN,NaN,NaN,NaN,NaN
7,enoxaparin,anticoagul|aspirin|lovenox|enoxaparin|heparin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,he couldn't tolerate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,heparin,anticoagul|aspirin|lovenox|enoxaparin|heparin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# drop table if exists
drop_query: str = "DROP TABLE IF EXISTS noteevents_05012023"
Sql.drop_table(drop_query)

# create pandas dataframe from the data and apply regex function. Heavy operation. Took 30 mins to complete.
query = "SELECT * FROM noteevents"
_df = Sql.select_db(query)

df_dd = applyRegEx(_df, 'text')

# write back to the database. Heavy operation. Took 30 mins to complete.
table: str = "noteevents_05012023"
Sql.write_df_to_db(df_dd, table)